In [9]:
import requests
import pandas as pd
from tqdm import tqdm
import os

In [10]:
# ======================
# Load district mapping
# ======================
df_map = pd.read_csv("district_mapping.csv", dtype={'id': str})
df_map['name'] = df_map['name'].astype(str).str.strip()

In [11]:
# ======================
# Create output folder
# ======================
OUTPUT_DIR = "district-weather-data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [12]:
# ======================
# API info
# ======================
url = "https://official.airbkk.com/airbkk/Report/getData"
PARAMS = ["PM2.5", "WS", "WD", "Temp", "RH", "BP"]
date_s = "01/01/2023"
date_e = "31/12/2024"

In [13]:
def convert_be(d):
    """Convert Buddhist Era to Christian Era"""
    try:
        day, month, year = d.split("/")
        year = str(int(year) - 543)
        return f"{year}-{month}-{day}"
    except:
        return d

In [14]:
# ======================
# Loop through districts
# ======================
for _, row in tqdm(df_map.iterrows(), total=df_map.shape[0]):

    meas_index = row['id']
    district_name = row['name']

    payload = {
        "groupid": "all",
        "MeasIndex[]": meas_index,
        "parameterTags[]": PARAMS,
        "data_type": "daily",
        "date_s": date_s,
        "date_e": date_e,
        "display_type": "table",
    }

    try:
        r = requests.post(url, data=payload)
        r.raise_for_status()

        data = r.json()
        arr = data.get("arrData", [])

        if len(arr) == 0:
            print(f"No data for {district_name} (ID {meas_index})")
            continue

        df_out = pd.DataFrame(arr)

        # Fix date BE → CE
        df_out["Date_Time"] = df_out["Date_Time"].apply(convert_be)

        # Add district column
        df_out["district"] = district_name

        # Rename columns
        new_col_map = {}
        for par in data.get("arrParameter", []):
            alias = par["Alias"]
            name = par["ShortName"]
            new_col_map[alias] = name

        df_out = df_out.rename(columns=new_col_map)

        # Ensure required columns exist
        for col in PARAMS:
            if col not in df_out.columns:
                df_out[col] = None

        # Final column order
        final_cols = ["district", "Date_Time"] + PARAMS
        df_out = df_out[final_cols]

        # ================================
        # Save into district-weather-data
        # ================================
        filename = f"{district_name}_weather_data.csv"
        filepath = os.path.join(OUTPUT_DIR, filename)

        df_out.to_csv(filepath, index=False, encoding="utf-8-sig")

        print(f"✔ Saved: {filepath}")

    except Exception as e:
        print(f"Error for {district_name} (ID {meas_index}): {e}")

  1%|          | 1/82 [00:01<02:18,  1.71s/it]

✔ Saved: district-weather-data\คลองสาน_weather_data.csv


  2%|▏         | 2/82 [00:03<02:20,  1.75s/it]

✔ Saved: district-weather-data\คลองสามวา_weather_data.csv


  4%|▎         | 3/82 [00:05<02:24,  1.83s/it]

...

✔ Saved: district-weather-data\ยานนาวา (สถานี คพ.)_weather_data.csv


 99%|█████████▉| 81/82 [03:16<00:01,  1.26s/it]

✔ Saved: district-weather-data\วังทองหลาง (สถานี คพ.)_weather_data.csv


100%|██████████| 82/82 [03:18<00:00,  2.42s/it]

✔ Saved: district-weather-data\วังทองหลาง (สถานี คพ.)_weather_data.csv
